# run_all_analyses.py

Here, we can combine our code into one code file. This code is not meant to be run here in this notebook, just to combine the code. When it is done we can copy paste the code to a .py file and run it to check whether everything works.

I put the functions in the first block and the rest of the code in the second block.

This code assumes that the folder "data" contains: 
* eng/abortion_overview_clean.tsv
* nld/abortus_overview_clean.tsv
* mrc2.dct.txt
* concreteness.xlsx
* AoA.xlsx
* wiki-news-300d-1M.vec
* cc.nl.300.vec

In [ ]:
# run_all_analyses.py
# Roderick Li and Jasmijn Cnossen
# Language as Data
# December 2021

"""
This script will run several semantic and stylistic analyses on our English and Dutch datasets.

First, we will define some functions to make the code more clear.
You can scroll down to were our analyses start.
It will take approximately ... minutes to run this script
"""

# import all packages
import time
import pandas as pd
import numpy as np
import stanza
import string
import statistics
from collections import Counter, OrderedDict
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import nltk
nltk.download('alpino')
from nltk.corpus import alpino
from sklearn.cluster import KMeans
import random
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
import torch
from transformers import AutoTokenizer, AutoModel
# random.seed(10)

# Load tokenizer from NLTK
nltk.download('punkt')
from nltk import word_tokenize, sent_tokenize
# If the code above bugs out, install additional dependencies, or use the following code:
# from nltk.tokenize import ToktokTokenizer (naive tokenizer)
# import re
# sent_tokenize = lambda x: re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', x)
# toktok = ToktokTokenizer()
# word_tokenize = toktok.tokenize

# pd.set_option("display.max_rows", None, "display.max_columns", None)

# We keep track of the time it takes to run the code:
start = time.perf_counter()

########################
### DEFINE FUNCTIONS ###
########################

### PREPROCESSING ###

# functions for reading in the data, tokenizing it with stanza, apply preprocessing, remove infrequent words

def tokenize_articles(dataframe, iso1):
    """
    dataframe: pandas dataframe with articles
    iso1: "en" or "nl"

    returns list of documents tokenized with stanza
    """
    articles = dataframe["Text"]
    
    # load stanza tokenizer
    stanza.download(iso1)
    tokenizer = stanza.Pipeline(iso1, processors='tokenize')
    
    # tokenize articles
    tokenized_articles = []
    for article in articles:
        tokenized_articles.append(tokenizer.process(article))
    
    return tokenized_articles

def preprocess_articles(articles, language):
    """
    articles: list of articles tokenized by stanza
    language: "english" or "dutch"

    returns list of articles with list of sentences with list of filtered and preprocessed words
    """
    stop_words = set(stopwords.words(language))
    
    preprocessed_articles = []
    for article in articles:
        all_tokens = []
        # get lowercase tokens without stopwords or punctuation:
        for sentence in article.sentences:
            tokens_sent = [token.text.lower() for token in sentence.tokens if not token.text.lower() in stop_words]
            all_punctuation = string.punctuation + "“”—’‘„"
            clean_tokens = [token for token in tokens_sent if not token in all_punctuation]
            all_tokens.append(clean_tokens)
        preprocessed_articles.append(all_tokens)
    
    return preprocessed_articles

def filter_rare_words(docs, frequency_threshold):
    """
    This functions filters words occuring less than the frequency threshold.
    docs: list of articles with list of sentences with list of tokenized words
    """
    alltokens = [token for doc in docs for sentence in doc for token in sentence]

    # Count frequency of tokens and determine the infrequent ones:
    kw_counter = Counter(alltokens)
    infrequent_tokens = Counter({k: c for k, c in kw_counter.items() if c < frequency_threshold})
    
    # filtering out the infrequent words
    frequent_docs = []
    for doc in docs:
        sentences_2 = []
        for sentence in doc:
            tokens_2 = []
            for token in sentence:
                if token not in infrequent_tokens.keys():
                    tokens_2.append(token)
            sentences_2.append(tokens_2)
        frequent_docs.append(sentences_2)
    
    return frequent_docs


### GETTING EMBEDDING FOR WORDS ###
def featureVecMethod(list_of_sents, # A list of tokenized sentences
                     model, # The actual word embeddings model
                     modelword_index, # An index on the vocabulary of the model to speed up lookup
                     num_embedding_dimensions # the number of dimensions of the embedding model
                    ):
    list_of_words = []
    for sent in list_of_sents:
        list_of_words.extend(sent)

    # Pre-initialising empty numpy array (np) for speed
    # This create a numpy array with the length of the num_features set to zero values
    featureVec = np.zeros(num_embedding_dimensions,dtype="float32")

    ## A counter for the number of tokens represented so that we can take the average
    nwords = 0
    embedding_words = []
    no_embeddings_words = []

    for word in list_of_words:
        if word in modelword_index:
            ### Instead of simply taking the embedding values we prefer the next function that
            ### normalises these values between [-1, 1] to make the average work better
            featureVec = np.add(featureVec,model[word]/np.linalg.norm(model[word]))

            # keep track of the words detected, just for analysing the data
            embedding_words.append(word)
            nwords = nwords + 1
        else:
            # keep track of the unknown words to see how well our model fits the data
            no_embeddings_words.append(word)

    # Dividing the result by number of words in each utterance to get average
    if nwords>0:
        featureVec = np.divide(featureVec, nwords)

    return featureVec, embedding_words, no_embeddings_words

# Function for calculating the average feature vector
def getAvgFeatureVecs(docs, ### List of documents of lists of sentences
                      model,
                      modelword_index,
                      num_embedding_dimensions
                     ):
    counter = 0
    embedding_words=[]
    no_embeddings_words=[]

    textFeatureVecs = np.zeros((len(docs),num_embedding_dimensions),dtype="float32")
    print('Shape of our matrix is:',textFeatureVecs.shape)

    #### Iterate over all the texts
    for doc in docs:
        # Printing a status message every 1000th text, to see what we are processing
        if counter%20 == 0:
            print("Vectorizing %d of %d"%(counter,len(docs)))
            # Checkpoint
        ### Each text is transformed into a vector representation based on the averaged token embedding using the previous function
        ### Add these vectors to the total matrix
        textFeatureVecs[counter], emb_words, nemb_words = featureVecMethod(doc, model, modelword_index,num_embedding_dimensions)
        counter = counter+1
        embedding_words.append(emb_words)
        no_embeddings_words.append(nemb_words)

    #### Due to the averaging, there could be infinitive values or NaN values. The next numpy function turns these value to "0" scores
    textFeatureVecs = np.nan_to_num(textFeatureVecs)

    return textFeatureVecs, embedding_words, no_embeddings_words

### PLOTTING IN TWO DIMENSIONS ###
# Reduce vector dimensions
def reduce_dimensions(word_vectors, size_vocab):
    model = word_vectors

    size_vocab = size_vocab
    vocab = [model.index_to_key[i] for i in range(size_vocab)]

    high_dimensional = [model[w] for w in vocab]
    reduction_technique = TSNE(n_components=2)

    print("Calculate dimensionality reduction")
    two_dimensional = reduction_technique.fit_transform(high_dimensional)
    print("Done")
    return two_dimensional

# Get the indices for most frequent terms
def get_index_for_frequent_terms(known_words, word_vectors, size_vocab):
    frequencies = Counter()

    for doc in known_words:
        frequencies.update(doc)

    # find 30 most frequent terms
    frequencies_top30 = OrderedDict(frequencies.most_common(30))
    terms = list(frequencies_top30.keys())
    print(terms)

    term_indices = [word_vectors.key_to_index[t] for t in terms]
    # print(term_indices)

    # remove terms + their indices that are not in the loaded vocabulary (size_vocab = 12000, see previous cell)
    remove = []
    for i, index in enumerate(term_indices):
        if index > size_vocab:
            remove.append(i)

    length = len(terms)
    terms = [terms[i] for i in range(length) if i not in remove]
    term_indices = [term_indices[i] for i in range(length) if i not in remove]

    return terms, term_indices

def plot_embedding_scatter(terms, term_indices, two_dimensional, iso1):
    fig, ax = plt.subplots(1, 1, figsize = (15, 10))

    # Plot the two-dimensional vectors for the selected terms
    x_values = [two_dimensional[index, 0] for index in term_indices] # if index < 12000
    y_values = [two_dimensional[index, 1] for index in term_indices]

    ax.plot(x_values, y_values, 'o')

    if iso1 == "en":
        color = 'm'
        lang = 'English'
    if iso1 == "nl":
        color = 'c'
        lang = 'Dutch'

    for x, y in zip(x_values, y_values):
        ax.plot(x, y, 'o', markersize=12, color=color)

    ax.set_title(f'2D word vectors for most frequent {lang} terms', fontsize=20)

    ax.set_yticks([])
    ax.set_xticks([])

    # Annotate the terms in the plot
    for i, word in enumerate(terms):
        plt.annotate(word, xy=(x_values[i], y_values[i]), fontsize=14)

    plt.show()
  ###

### MAKING THE WORD CLOUDS ###
# Use word cloud to evaluate the cluster
def wordcloud_cluster_byIds(clusterId, clusters, doc):
    words = []
    for i in range(0, len(clusters)):
        if clusters[i] == clusterId:
            for word in doc[i]:
                words.append(word)
    # Only printing the results I want:
    # if len(words)==
    print("Nr of words in cluster:", len(words))
    print("Samples", words[::max((len(words)//10), 1)])
    # Generate a word cloud based on the frequency of the terms in the cluster
    wordcloud = WordCloud(max_font_size=40, relative_scaling=.8).generate(' '.join(words))

    plt.figure()
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.savefig(str(clusterId)+".png")

def making_word_clouds(vectors, num_clusters, known_words, random):
    km = KMeans(n_clusters=num_clusters, random_state=random)
    km.fit(vectors)

    clusters = km.labels_.tolist()
    for ind in range(0, num_clusters):
        wordcloud_cluster_byIds(ind, clusters, known_words)
    return clusters

def map_cluster_to_source(cluster_list, iso1):
    # Mapping the clusters to metadata
    if iso1=="en":
        news_content = news_content_eng
    if iso1=="nl":
        news_content = news_content_nld
    clustered_articles ={'Title': news_content["Title"],'Author': news_content["Author"],'Source': news_content["Source"], 'Cluster': cluster_list}
    overview = pd.DataFrame(clustered_articles, columns = ['Author', 'Title', 'Source', 'Cluster'])
    for num in set(cluster_list):
        cluster = overview[overview['Cluster']==num]

        # Counting the sources of each clusters
        print(f"Info of cluster {num+1}")
        print(cluster.info())
        print(f"Source count of cluster {num+1}:")
        print(cluster['Source'].value_counts())
###

### LOADING WORD FEATURES FROM DATABASES ###

# function to load mrc database
def load_mrc(file_name):
    """
    function that loads the mrc database and extracts features
    returns dictionary with Brown frequency, familiarity, concreteness and age of acquisition for each word
    """
    words ={}
    print("Loading mrc...")
    for line in open(file_name,'r'):
        
        # This code is from https://github.com/samzhang111/mrc-psycholinguistics/blob/master/extract.py
        word, phon, dphon, stress = line[51:].split('|')

        nlet = int(line[0:2])
        nphon = int(line[2:4])
        nsyl = int(line[4])
        kf_freq = int(line[5:10])
        kf_ncats = int(line[10:12])
        kf_nsamp = int(line[12:15])
        tl_freq = int(line[15:21])
        brown_freq = int(line[21:25])
        fam = int(line[25:28])
        conc = int(line[28:31])
        imag = int(line[31:34])
        meanc = int(line[34:37])
        meanp = int(line[37:40])
        aoa = int(line[40:43])
        tq2 = line[43]
        wtype = line[44]
        pdwtype = line[45]
        alphasyl = line[46]
        status = line[47]
        var = line[48]
        cap = line[49]
        irreg = line[50]
       
        words[word.lower()] = (int(brown_freq), int(conc), int(aoa))
    print("Done.")
    return words

# calculating Dutch features: word frequencies, concreteness, age of acquisition
def alpino_frequency():
    """
    function that provides a dictionary of all words and their frequency in the Alpino corpus (Dutch)
    """
    print("loading alpino frequency...")
    corpus = " ".join([word.lower() for word in alpino.words()])
    tokenized_corpus = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(corpus)]
    tokenized_corpus = tokenized_corpus[0]
    Alpino_counter = Counter(tokenized_corpus)
    print("Done.")
    
    return Alpino_counter

def concreteness_nld(file_name, sheet):
    """
    function that provides a dictionary of Dutch words and their concreteness
    ratings from: http://crr.ugent.be/archives/1602
    """
    print("loading concreteness nld...")
    df = pd.read_excel(io=file_name, sheet_name=sheet, header=0)
    conc_dict = dict([(word, conc) for word, conc in zip(df.stimulus, df.Concrete_m)])
    print("Done.")
    
    return conc_dict

def aoa_nld(file_name):
    """
    function that provides a dictionary of Dutch words and their age of acquisition
    ratings from: http://crr.ugent.be/archives/1602
    """
    print("loading aoa nld...")
    df = pd.read_excel(io=file_name, header=0, skiprows=[1])
    aoa_dict = dict([(word, aoa) for word, aoa in zip(df.Word, df.Average)])
    print("Done.")
    
    return aoa_dict


### CALCULATE FEATURE VECTORS ###

# Functions that calculates average of a feature over tokens for English and Dutch, using the databases defined above
def calculate_eng_features(tokens, mrc, n):
    """
    tokens = list of tokens
    mrc = dictionary containing several features for each word in mrc database
    n = nr of features outputted by mrc (length tuple (value) for each word (key))
    
    returns list of length n of the average rating over the tokens for each of the n mrc_features
    """
    
    feature_lists = [[] for i in range(n)]
    stats = []
    
    for token in tokens:
        # For words that are not in the database, we assign the rating 0
        feature_ratings = mrc.get(token.lower(), (0,0)) # returns tuple with ratings for all specified features for this token
        
        # We only consider words that have a rating in the database
        for i, rating in enumerate(feature_ratings):
            if rating > 0:
                feature_lists[i].append(rating)
             
    # Take the mean over all tokens for every feature and append to stats:
    for feature in feature_lists:
        if len(feature) > 0:
            stats.append(statistics.mean(feature))
        else:
            stats.append(0.0)
    
    return stats

def calculate_nld_features(tokens, alpino_counter, conc_dict, aoa_dict):
    """
    tokens = list of tokens
    alpino_counter = dictionary containing frequency for each word in alpino corpus
    conc_dict = dictionary containing concreteness ratings of Dutch words
    aoa_dict = dictionary containing age of acquisition ratings of Dutch words
    
    returns the average of each feature over the tokens
    """
    feature_lists = [[] for i in range(3)]
    stats = []
    
    for token in tokens:
        # For words that are not in the database, we assign the rating 0
        freq = alpino_counter.get(token.lower(), 0)
        conc = conc_dict.get(token.lower(), 0)
        aoa = aoa_dict.get(token.lower(), 0)
        
        # We only consider words that have a rating in the database
        for i, rating in enumerate([freq, conc, aoa]):
            if rating > 0:
                feature_lists[i].append(rating)
    
    # Take the mean over all tokens for every feature and append to stats:
    for feature in feature_lists:
        if len(feature) > 0:
            stats.append(statistics.mean(feature))
        else:
            stats.append(0.0)
    
    return stats

# Function that turns articles into feature vectors:
def feature_vectorize(docs, language, mrc, alpino_counter, conc_dict, aoa_dict):
    """
    docs: list of articles with list of sentences with list of tokens
    language: "eng" or "nld"
    mrc: for language = "eng"
    alpino_counter, conc_dict, aoa_dict: for language = "nld"
    
    returns vector representation for each article based on several defined features.
    """

    all_docs =[]

    for article in docs:
        doc_representation = []

        # Calculate TTR + average sentence length + average token length
        token_frequencies = Counter()
        sentence_lengths = []
        token_lengths = []
        for sentence in article:
            tokens_sent = [token for token in sentence]
            token_frequencies.update(tokens_sent)
            sentence_lengths.append(len(sentence))
            token_lengths = token_lengths + [len(token) for token in sentence]
        num_types = len(token_frequencies.keys())
        num_tokens = sum(token_frequencies.values())
        tt_ratio = num_types/float(num_tokens)
        doc_representation.append(tt_ratio) # [0] type-token ratio
        doc_representation.append(statistics.mean(sentence_lengths)) # [1] avg sentence length
        doc_representation.append(statistics.mean(token_lengths)) # [2] avg token length

        if language == "eng":
            # Calculate additional features
            tokens = [word for sentence in article for word in sentence]
            stats = calculate_eng_features(tokens, mrc, 3)
            doc_representation.append(stats[0]) # [3] Brown frequency
            doc_representation.append(stats[1]) # [4] concreteness
            doc_representation.append(stats[2]) # [5] age of acquisition
            
        if language == "nld":
            # Calculate additional features
            tokens = [word for sentence in article for word in sentence]
            stats = calculate_nld_features(tokens, alpino_counter, conc_dict, aoa_dict)
            doc_representation.append(stats[0]) # [3] Alpino frequency
            doc_representation.append(stats[1]) # [4] concreteness
            doc_representation.append(stats[2]) # [5] age of acquisition

        all_docs.append(doc_representation)
    
    return all_docs

### CLUSTERING BASED ON FEATURE VECTORS ###

# function that standardizes the feature vectors
def standardize_array(array):
    """
    array: instances on rows, features on columns
    returns and array with z-scored values based on the mean and std of the columns
    """

    array_standardized = np.zeros(array.shape)
    Means = []
    SDs = []
    
    # first, calculate the mean and standard deviation for all features
    # we need these for the z-scoring later
    n_features =  np.size(array, 1)
    for i in range(n_features):
        mean = statistics.mean(array[:,i])
        Means.append(mean)
        sd = statistics.stdev(array[:,i])
        SDs.append(sd)

    # standardizing (i.e. z-scoring) all values in all_stats and adding them to the new array
    for i, row in enumerate(array):
        for j, value in enumerate(row):
            z_score = (value - Means[j]) / SDs[j]
            array_standardized[i, j] = z_score

    # array_standardized contains the z-scored features, i.e. M = 0, SD = 1
    return array_standardized

# function that calculates the average over the features within each cluster
def avg_features_clusters(feature_vecs, cluster_list):
    """
    function that calculates the average over the features within each cluster
    
    feature_vecs: document vectors as numpy array
    n_clusters: number of clusters
    cluster_list: list of clusters for all articles
    """
    n_clusters = len(set(cluster_list))
    avg_features_per_cluster = []
    clusters = np.array(cluster_list)
    # for each cluster, calculate average over features
    for i in range(n_clusters):
        # get indices of articles within this cluster
        indices_cluster = list(np.where(clusters == i)[0])
        # select these articles in feature_vecs and calculate the average over the columns (i.e. features)
        feature_vecs_cluster = feature_vecs[indices_cluster]
        means = np.mean(feature_vecs_cluster, axis=0)
        avg_features_per_cluster.append(list(np.around(means, decimals = 3)))

    return avg_features_per_cluster

In [ ]:
#############################################
### READING IN THE DATA AND PREPROCESSING ###
#############################################

# Read in the English tsv file as a pandas dataframe
filepath = "data/eng/abortion_overview_clean.tsv"
news_content_eng = pd.read_csv(filepath, sep="\t", header = 0, keep_default_na=False)[:100]
# Tokenize the articles
tokenized_articles_eng = tokenize_articles(news_content_eng, "en")
# Preprocess the articles
docs_eng = preprocess_articles(tokenized_articles_eng, "english")

# Read in the Dutch tsv file as a pandas dataframe
filepath = "data/nld/abortus_overview_clean.tsv"
news_content_nld = pd.read_csv(filepath, sep="\t", header = 0, keep_default_na=False)[:100]
# Tokenize the articles
tokenized_articles_nld = tokenize_articles(news_content_nld, "nl")
# Preprocess the articles
docs_nld = preprocess_articles(tokenized_articles_nld, "dutch")

# Filter infrequent words (noise)
frequency_threshold = 2
docs_eng_filtered = filter_rare_words(docs_eng, frequency_threshold)
docs_nld_filtered = filter_rare_words(docs_nld, frequency_threshold)

### BASIC STATS ###

# Calculate average number of sentences of the articles
nr_sentences_per_article = []
for article in docs_eng:
    nr_sentences_per_article.append(len(article))
print("Average nr of sentences per article (English): ", np.mean(nr_sentences_per_article))

nr_sentences_per_article = []
for article in docs_nld:
    nr_sentences_per_article.append(len(article))
print("Average nr of sentences per article (Dutch): ", np.mean(nr_sentences_per_article))

# Calculate average number of words of the articles
nr_words_per_article = []
for article in docs_eng_filtered:
    nr_words = 0
    for sentence in article:
        nr_words += len(sentence)
    nr_words_per_article.append(nr_words)
print("Average nr of words per article (English): ", np.mean(nr_words_per_article))

nr_words_per_article = []
for article in docs_nld_filtered:
    nr_words = 0
    for sentence in article:
        nr_words += len(sentence)
    nr_words_per_article.append(nr_words)
print("Average nr of words per article (Dutch): ", np.mean(nr_words_per_article))

# Number of different sources + top 3
print("Nr. of different sources: ", news_content_eng["Source"].nunique())
print("Top 3 most occuring sources: \n", news_content_eng["Source"].value_counts()[:3].sort_values(ascending=False)) # most popular

print("Nr. of different sources: ", news_content_nld["Source"].nunique())
print("Top 3 most occuring sources: \n", news_content_nld["Source"].value_counts()[:3].sort_values(ascending=False)) # most popular

#########################
### SEMANTIC ANALYSES ###
#########################

### ENGLISH ###

# Loading the vector model: eng
print("loading English vector model")
word_vectors_eng = KeyedVectors.load_word2vec_format("data/wiki-news-300d-1M.vec")
print("done loading")

## Getting the word vectors
#Converting Index2Word which is a list to a set for better speed in the execution.
#Allows for quicker lookup if the words exist
index2key_set_eng = set(word_vectors_eng.index_to_key)
num_features = 300
# Calculating average vector for Eng set
eng_vectors, eng_known_words, eng_unknown_words = getAvgFeatureVecs(docs_eng_filtered, word_vectors_eng, index2key_set_eng, num_features)

# Plotting the English frequent words in two dimensions
size_vocab = 12000
two_dimensional_eng = reduce_dimensions(word_vectors_eng, size_vocab)
terms_eng, term_indices_eng = get_index_for_frequent_terms(eng_known_words, word_vectors_eng, size_vocab)
plot_embedding_scatter(terms_eng, term_indices_eng, two_dimensional_eng, "en")

## Clustering
# We will try 2, 3 and 4 clusters to see the difference
# Each number of clusters will be performed 3 times

# Defining a random seed
random.seed(0)

# Testing k number 2, 3 or 4, each run 3 times
for k in [2, 3, 4]:
    for i in range(3):
        state = random.randint(0, 42)
        # Cluster
        # clusters = making_word_clouds(eng_vectors, k, eng_known_words, state)
        # map_cluster_to_source(clusters, 'en')

        # After observation, only printing the results in the analysis
        if k==4 and i==1:
            clusters = making_word_clouds(eng_vectors, k, eng_known_words, state)
            map_cluster_to_source(clusters, 'en')

### DUTCH ###

# Loading the vector model: nld
print("loading Dutch vector model")
word_vectors_nld = KeyedVectors.load_word2vec_format("data/cc.nl.300.vec")
print("done loading")

## Getting the word vectors
# Converting Index2Word which is a list to a set for better speed in the execution.
# Allows for quicker lookup if the words exist
index2key_set_nld = set(word_vectors_nld.index_to_key)
num_features = 300
# Calculating average vector for training set
nld_vectors, nld_known_words, nld_unknown_words = getAvgFeatureVecs(docs_nld_filtered, word_vectors_nld, index2key_set_nld, num_features)

# Plotting the frequent Dutch terms in 2D
size_vocab = 12000
two_dimensional_nld = reduce_dimensions(word_vectors_nld, size_vocab)
terms_nld, term_indices_nld = get_index_for_frequent_terms(nld_known_words, word_vectors_nld, size_vocab)
plot_embedding_scatter(terms_nld, term_indices_nld, two_dimensional_nld, "nl")

## Clustering
# Defining a random seed
random.seed(42)

# Testing k number 2, 3 or 4, each run 3 times
for k in [2, 3, 4]:
    for i in range(3):
        state = random.randint(0, 42)
        # Cluster
        # clusters = making_word_clouds(nld_vectors, k, nld_known_words, state)
        # map_cluster_to_source(clusters, "nl")

        # After observation, only printing the results in the analysis
        if k==3 and i==0:
            clusters = making_word_clouds(nld_vectors, k, nld_known_words, state)
            map_cluster_to_source(clusters, "nl")

## CROSSLINGUAL BERT ##

# Initialize BERT
model_name = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Concatenating the documents
all_sents = []

for each in (news_content_eng, news_content_nld):
    for doc in list(each["Text"]):
        sent = doc.split(".")
        for s in sent:
            s = s + "."
            all_sents.append(s)

# Vectorization
vectors = []
all_tokens = []
print("Processing...")
for sent in all_sents:
    tokens = tokenizer.encode(sent, add_special_tokens=True)
    sent = [tokenizer.decode([token]) for token in tokens]
    all_tokens.append(sent)
    model.eval()  # turn off dropout layers
    tokens_tensor = torch.tensor(tokens).unsqueeze(0)
    output = model(tokens_tensor)
    vector = output[0].detach().numpy()[0]
    vectors.append(vector)
    if len(vectors)%1000==0:
        print(f"Processing...{len(vectors)}/{len(all_sents)}")
        print("Checkpoint")
        print(sent)
        print(vector.shape)
print("Done.")

high_dimensional_tok = []
high_dimensional_vec = []
# Randomly choose tokens to reduce dimension: use all the tokens in every 3 sentences if it is longer than 20 words
for sent in all_tokens[::3]:
    if len(sent)>20:
        high_dimensional_tok.extend(sent[1:-1])
        high_dimensional_vec.extend(vectors[all_tokens.index(sent)][1:-1])

reduction_technique = TSNE(n_components=2)

print("Calculate dimensionality reduction")
two_dimensional = reduction_technique.fit_transform(high_dimensional_vec)
print("Done")

terms = []
term_indices = []
for terms_list in (terms_eng, terms_nld):
    for term in terms_list:
        # If the frequent term is in the reduced dimension:
        if term in high_dimensional_tok:
            terms.append(term)
            term_indices.append(high_dimensional_tok.index(term))

fig, ax = plt.subplots(1, 1, figsize = (15, 10))

#term_indices = [high_dimensional.index[vec] for vec in term_vec]

# Plot the two-dimensional vectors for the selected terms
x_values = [two_dimensional[i, 0] for i in term_indices]
y_values = [two_dimensional[i, 1] for i in term_indices]

ax.plot(x_values, y_values, 'o')

for x, y in list(zip(x_values, y_values))[:20]:
    ax.plot(x, y, 'o', markersize=12, color='m')
for x, y in list(zip(x_values, y_values))[20:]:
    ax.plot(x, y, 'o', markersize=12, color='c')

ax.set_title(f'2D word vectors for most frequent English and Dutch terms', fontsize=20)

# Hide the ticks
ax.set_yticks([])
ax.set_xticks([])

# Annotate the terms in the plot
for i, word in enumerate(terms):
    plt.annotate(word, xy=(x_values[i], y_values[i]), fontsize=14)

plt.show()

##########################
### STYLISTIC ANALYSES ###
##########################

random.seed(10)

# Here we load the English features from the mrc database: Brown frequency, concreteness and age of acquisition (aoa)
file_name = 'data/mrc2.dct.txt'
mrc = load_mrc(file_name)

# Here, we load the Dutch features: Alpino frequency, concreteness and age of acquisition (aoa)
alpino_counter = alpino_frequency()

file_name =  'data/concreteness.xlsx'
sheet = 'FULLIST'
conc_dict = concreteness_nld(file_name, sheet)

file_name =  'data/AoA.xlsx'
aoa_dict = aoa_nld(file_name)

### ENGLISH ARTICLES ###

# We turn the English articles into feature vectors
# Each article is represented as a vector with 6 dimensions:
# Type-token ratio, avg sentence length, avg token length, avg frequency, avg concreteness and avg age of acquisition
doc_vectors_eng = feature_vectorize(docs_eng_filtered, "eng", mrc, alpino_counter, conc_dict, aoa_dict)
# print(doc_vectors_eng[:3])

# Calculate average of the features over all articles
print("Feature vector consists of: [TTR, avg sentence len, avg token len, freq, conc, AoA]")
print("Average feature vector over all English articles: \n", np.mean(doc_vectors_eng, axis=0))

# For k-means, it is important to standardize all features, so that they all contribute equally to the clustering.
doc_vectors_eng_stand = standardize_array(np.array(doc_vectors_eng))

# We cluster the articles into clusters with k = 2, k = 3 and k = 4
# And we repeat this 5 times to check how consistent the results are
for k in [2,3,4]:
    for i in range(5):
        # Cluster
        km = KMeans(n_clusters = k)
        km.fit(doc_vectors_eng_stand)
        
        # print how many articles per cluster
        clusters_eng = km.labels_.tolist()
        counts = []
        for j in range(k):
            count = clusters_eng.count(j)
            counts.append(count)
            # If we have a cluster with just one article, we print the outlier
            if count == 1:
                print(f"article {clusters_eng.index(j)} is an outlier")
#         print(counts)
        
        # Calculate the average feature vectors per cluster
#         print(avg_features_clusters(doc_vectors_eng_stand, k, clusters_eng))

# We choose k = 2 for further analyses, because we want to divide the articles up into "difficult" and "easy"

# Now we cluster the articles into 2 clusters
# Keep in mind that Python has 0 indexing, so cluster 0 will correspond to cluster 1 in our paper.
k = 2
km = KMeans(n_clusters = k)
km.fit(doc_vectors_eng_stand)

# Add the clusters as a column in our dataframe
clusters_eng = km.labels_.tolist()
clustered_articles ={'Title': news_content_eng["Title"],'Author': news_content_eng["Author"],'Source': news_content_eng["Source"], 'Cluster': clusters_eng}
overview_eng = pd.DataFrame(clustered_articles, columns = ['Title', 'Author', 'Source', 'Cluster'])
# overview_eng.sort_values(['Cluster', 'Source'])

# count articles
print("Cluster counts:")
print(overview_eng["Cluster"].value_counts())

# Count articles per source per cluster
print(overview_eng.groupby("Source")["Cluster"].value_counts())

# We calculate the average of each feature within each cluster
# We use the standardized feature vectors because then we can compare the differences
print("Average feature vectors within each cluster (English)")
print("[TTR, avg sent length, avg token length, freq, conc, AoA]")
print(avg_features_clusters(doc_vectors_eng_stand, clusters_eng))

### DUTCH ARTICLES ###

# We turn the Dutch articles into feature vectors
# Each article is represented as a vector with 6 dimensions:
# Type-token ratio, avg sentence length, avg token length, avg frequency, avg concreteness and avg age of acquisition
doc_vectors_nld = feature_vectorize(docs_nld_filtered, "nld", mrc, alpino_counter, conc_dict, aoa_dict)
# print(doc_vectors_nld[:3])

# Calculate average of the features over all articles
print("Feature vector consists of: [TTR, avg sentence len, avg token len, freq, conc, AoA]")
print("Average feature vector over all Dutch articles: \n", np.mean(doc_vectors_nld, axis=0))

# For k-means, it is important to standardize all features, so that they all contribute equally to the clustering.
doc_vectors_nld_stand = standardize_array(np.array(doc_vectors_nld))

# We cluster the articles into clusters with k = 2, k = 3 and k = 4
# And we repeat this 5 times to check how consistent the results are
for k in [2,3,4]:
    for i in range(5):
        # Cluster
        km = KMeans(n_clusters = k)
        km.fit(doc_vectors_nld_stand)
        
        # print how many articles per cluster
        clusters_nld = km.labels_.tolist()
        counts = []
        for j in range(k):
            count = clusters_nld.count(j)
            counts.append(count)
            # If we have a cluster with just one article, we print the outlier
            if count == 1:
                print(f"article {clusters_nld.index(j)} is an outlier")
        # print(counts)
        
        # Calculate the average feature vectors per cluster
        # print(avg_features_clusters(doc_vectors_nld_stand, clusters_nld))

# We choose k = 2 for further analyses, because we want to divide the articles up into "difficult" and "easy"

# Now we cluster the articles into 2 clusters
# Keep in mind that Python has 0 indexing, so cluster 0 will correspond to cluster 1 in our paper.
k = 2
km = KMeans(n_clusters = k)
km.fit(doc_vectors_nld_stand)

# Add the clusters as a column in our dataframe
clusters_nld = km.labels_.tolist()
clustered_articles ={'Title': news_content_nld["Title"],'Author': news_content_nld["Author"],'Source': news_content_nld["Source"], 'Cluster': clusters_nld}
overview_nld = pd.DataFrame(clustered_articles, columns = ['Author', 'Title', 'Source', 'Cluster'])
# overview_nld.sort_values(['Cluster', 'Source'])

# count articles
print("Cluster counts:")
print(overview_nld["Cluster"].value_counts())

# Count articles per source per cluster
print(overview_nld.groupby("Source")["Cluster"].value_counts())

# We calculate the average of each feature within each cluster
# We use the standardized feature vectors because then we can compare the differences
print("Average feature vectors within each cluster (Dutch)")
print("[TTR, avg sent length, avg token length, freq, conc, aoa]")
print(avg_features_clusters(doc_vectors_nld_stand, clusters_nld))


### CALCULATE THE TIME IT TOOK TO RUN ALL CODE ###
stop = time.perf_counter()
timer = stop - start
minutes = int(timer / 60)
print(f"Time it took to run the code: {minutes} minutes")

